# FGBG Seperation via Color Pixel Mask and Red Sequence Members 

### Imports

In [ ]:
from astropy.table import Table, vstack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy import ndimage
from matplotlib.colors import to_rgba
import matplotlib.cm as cm

### Load in Redshift Data and the Full Cluster Catalog

In [ ]:
# Data file paths
# Replace with your own /path/to/data
REDSHIFT_CATALOG = '/scratch/sa.saha/data/mega_color_mag_with_redshift_20250811_141642.fits'  # Training data with known redshifts
FULL_CATALOG = '/scratch/sa.saha/data/mega_color_mag_catalog_20250811_141158.fits'            # Full catalog

# Load the redshift catalog (for visualization and validation)
print("Loading redshift catalog for training/validation...")
redshift_cat = Table.read(REDSHIFT_CATALOG)
print(f"Loaded {len(redshift_cat)} objects with known redshifts")

# Load the full catalog (for applying cuts)
print("\nLoading full photometric catalog...")
full_cat = Table.read(FULL_CATALOG)
print(f"Loaded {len(full_cat)} total objects")

# Display available columns
print(f"\nAvailable columns: {full_cat.colnames}")

### Define Variables

In [ ]:
redshift = redshift_cat['Z_best'].astype(float)
color_bg = redshift_cat['color_bg'].astype(float)  # B-G color *includes lovoccs*
color_ub = redshift_cat['color_ub'].astype(float)  # U-B color
color_bg_err = redshift_cat['color_bg_err'].astype(float)
color_ub_err = redshift_cat['color_ub_err'].astype(float)
redshift_err = redshift_cat["ZERR_best"].astype(float)
redshift_source = np.array([s.strip() for s in redshift_cat['Z_source']])

color_bg_full = full_cat['color_bg'].astype(float)  # B-G color *includes lovoccs*
color_ub_full = full_cat['color_ub'].astype(float)  # U-B color
color_bg_err_full = full_cat['color_bg_err'].astype(float)
color_ub_err_full = full_cat['color_ub_err'].astype(float)


print("Data summary:")
print(f"Redshift range: {np.nanmin(redshift):.3f} to {np.nanmax(redshift):.3f}")
print(f"B-G color range: {np.nanmin(color_bg):.3f} to {np.nanmax(color_bg):.3f}")
print(f"U-B color range: {np.nanmin(color_ub):.3f} to {np.nanmax(color_ub):.3f}")
print(f"Redshift sources: {np.unique(redshift_source)}")

### Load in the red sequence data 


In [ ]:
# Red Sequence Members 
Abell3411_imposters = Table.read('/projects/mccleary_group/ysp/redseq/Abell3411/Abell3411_imposter_members_catalog.fits')
Abell3411_actual = Table.read('/projects/mccleary_group/ysp/redseq/Abell3411/Abell3411_actual_members_catalog.fits')

Bullet_imposters = Table.read('/projects/mccleary_group/ysp/redseq/1E0657_Bullet/1E0657_Bullet_imposter_members_catalog.fits')
Bullet_actual = Table.read('/projects/mccleary_group/ysp/redseq/1E0657_Bullet/1E0657_Bullet_actual_members_catalog.fits')

## Define Cluster and General Parameters

In [ ]:
Z_THRESH = 0.2
ERR_THRESH = 20
CLUSTER_NAME = 'Abell3411'
XLIM = (-1,3) #(-0.5, 3)    
YLIM = (-2, 3.5) #(-2, 3) 
PIXEL_SIZE = 0.05
PURITY_THRESHOLD = 0.8


In [ ]:
# Valid redshift sources (NED and DESI are most reliable)
reliable_sources_mask = (redshift_source == "NED") 

# Good color measurements (low errors)
good_colors_mask = (color_bg_err < ERR_THRESH) & (color_ub_err < ERR_THRESH)

# No NaN values in colors or redshift
valid_data_mask = ~(np.isnan(color_bg) | np.isnan(color_ub) | np.isnan(redshift))

# Combined mask for "training"
training_mask = reliable_sources_mask & good_colors_mask & valid_data_mask

# Background vs foreground classification
background_mask = redshift > Z_THRESH  # Objects behind the cluster
foreground_mask = redshift < Z_THRESH  # Objects in front of the cluster

# data quality stats
total_objects = len(redshift)
nan_objects = np.sum(~valid_data_mask)
high_error_objects = np.sum(~good_colors_mask & valid_data_mask)  # Valid data but high errors!
unreliable_sources = np.sum(~reliable_sources_mask)

## Functions

In [ ]:
def create_pixel_voting_map_purity(color_bg, color_ub, redshift, z_thresh, xlim, ylim,
                                   pixel_size, purity_threshold, training_mask=None,
                                   weighting=False, color_bg_err=None, color_ub_err=None):
    """
    Create a pixel-based voting map in color-color space with purity threshold.
    Each pixel is colored based on whether foreground (blue) objects meet the purity threshold.
    
    Parameters
    ----------
    color_bg, color_ub : array-like
        B-G and U-B colors
    redshift : array-like
        Redshift values
    z_thresh : float
        Redshift threshold for background/foreground split
    xlim, ylim : tuple
        Plot limits for color axes
    pixel_size : float
        Size of each pixel in color space
    purity_threshold : float
        Minimum fraction of foreground objects needed for pixel to be blue
    training_mask : boolean array, optional
        Mask for high-quality training data
    weighting : bool, optional
        If True, use variance-based weighting with color errors
    color_bg_err, color_ub_err : array-like, optional
        Color errors for B-G and U-B (required if weighting=True)
    
    Returns
    -------
    vote_map : 2D array
        Voting results: 1 for red (background-dominated or below purity),
        -1 for blue (foreground purity >= threshold)
    x_edges, y_edges : array
        Bin edges for the pixel grid
    bg_hist, fg_hist : 2D arrays
        Object counts (or weighted sums) for background and foreground populations
    total_objects : 2D array
        Total weighted sum (or counts if no weighting) per pixel
    actual_counts : 2D array
        Actual object counts per pixel (always unweighted)
    """
    
    # Handle weighting parameter validation
    if weighting and (color_bg_err is None or color_ub_err is None):
        raise ValueError("color_bg_err and color_ub_err must be provided when weighting=True")
    
    # Apply training mask if provided
    if training_mask is not None:
        color_bg = color_bg[training_mask]
        color_ub = color_ub[training_mask]
        redshift = redshift[training_mask]
        if weighting:
            color_bg_err = color_bg_err[training_mask]
            color_ub_err = color_ub_err[training_mask]
    
    # Remove NaN values
    valid_mask = ~(np.isnan(color_bg) | np.isnan(color_ub) | np.isnan(redshift))
    if weighting:
        valid_mask &= ~(np.isnan(color_bg_err) | np.isnan(color_ub_err))
    
    color_bg = color_bg[valid_mask]
    color_ub = color_ub[valid_mask]
    redshift = redshift[valid_mask]
    if weighting:
        color_bg_err = color_bg_err[valid_mask]
        color_ub_err = color_ub_err[valid_mask]
    
    # Create redshift masks
    background_mask = redshift > z_thresh  # High-z (background, RED)
    foreground_mask = redshift <= z_thresh  # Low-z (foreground, BLUE)
    
    # Create pixel grid
    x_edges = np.arange(xlim[0], xlim[1] + pixel_size, pixel_size)
    y_edges = np.arange(ylim[0], ylim[1] + pixel_size, pixel_size)
    
    # ALWAYS calculate actual object counts (unweighted)
    bg_counts, _, _ = np.histogram2d(color_bg[background_mask], color_ub[background_mask],
                                     bins=[x_edges, y_edges])
    fg_counts, _, _ = np.histogram2d(color_bg[foreground_mask], color_ub[foreground_mask],
                                     bins=[x_edges, y_edges])
    actual_counts = bg_counts + fg_counts
    
    # Calculate weights if requested
    if weighting:
        # Average of BG and UB errors
        combined_error = (color_bg_err + color_ub_err) / 2.0
        fg_errors = combined_error[foreground_mask]
        bg_errors = combined_error[background_mask]

        print(f"Foreground median error: {np.median(fg_errors):.3f}")
        print(f"Background median error: {np.median(bg_errors):.3f}")
        
        
        # Variance-based weighting: weight = 1/error²
        # Add small epsilon to avoid division by zero
        epsilon = 1e-10
        weights = 1.0 / (combined_error**2 + epsilon)

        print(f"Foreground mean weight: {np.mean(1/(fg_errors**2 + epsilon)):.3f}")
        print(f"Background mean weight: {np.mean(1/(bg_errors**2 + epsilon)):.3f}")
    

        # Check for extreme weights
        print(f"Max foreground weight: {np.max(1/(fg_errors**2 + epsilon)):.0f}")
        print(f"Max background weight: {np.max(1/(bg_errors**2 + epsilon)):.0f}")
        
        # Split weights by population
        bg_weights = weights[background_mask]
        fg_weights = weights[foreground_mask]
        
        # Create weighted histograms
        bg_hist, _, _ = np.histogram2d(color_bg[background_mask], color_ub[background_mask],
                                       bins=[x_edges, y_edges], weights=bg_weights)
        fg_hist, _, _ = np.histogram2d(color_bg[foreground_mask], color_ub[foreground_mask],
                                       bins=[x_edges, y_edges], weights=fg_weights)
    else:
        # No weighting - histograms are just the counts
        bg_hist = bg_counts
        fg_hist = fg_counts
        bg_weights = None
        fg_weights = None
    
    # Create voting map based on purity threshold
    vote_map = np.zeros_like(bg_hist)
    
    # Calculate total (weighted or unweighted) objects per pixel
    total_objects = bg_hist + fg_hist
    has_objects = total_objects > 0
    
    # Calculate (weighted) purity
    purity = np.zeros_like(bg_hist)
    purity[has_objects] = fg_hist[has_objects] / total_objects[has_objects]
    
    # Apply purity threshold: blue (-1) if purity >= threshold, red (+1) otherwise
    vote_map[has_objects] = np.where(purity[has_objects] >= purity_threshold, -1, 1)
    
    return vote_map, x_edges, y_edges, bg_hist, fg_hist, total_objects, actual_counts

In [ ]:
def plot_pixel_voting_map(color_bg, color_ub, redshift, z_thresh, xlim, ylim, 
                                training_mask, cluster_name, pixel_size, purity_threshold,
                                show_scatter=True, show_counts=True, min_count_display=1,
                                blue_counts_only=False, 
                                show_cluster_objects=False, full_catalog=None,
                                redshift_seq=False, imposters_catalog=None, actual_catalog=None,
                                show_mask=False, figsize=(10, 8),
                                weighting=False, color_bg_err=None, color_ub_err=None):
    """
    Plot pixel-based voting map with purity threshold and optional object counts.
    
    Parameters
    ----------
    [Previous parameters...]
    show_mask : bool
        If True, color BLUE pixels with >=10 objects green
    weighting : bool, optional
        If True, use variance-based weighting with color errors for purity calculation
    color_bg_err, color_ub_err : array-like, optional
        Color errors for B-G and U-B (required if weighting=True)
    """
    
    # Create voting map with purity threshold (now with optional weighting)
    vote_map, x_edges, y_edges, bg_hist, fg_hist, total_objects, actual_counts = create_pixel_voting_map_purity(
        color_bg, color_ub, redshift, z_thresh, xlim, ylim, 
        pixel_size, purity_threshold, training_mask,
        weighting=weighting, color_bg_err=color_bg_err, color_ub_err=color_ub_err
    )
    
    # Apply mask coloring if requested
    if show_mask:
        # Make a copy of vote_map to modify
        vote_map_display = vote_map.copy()
        
        # Find BLUE pixels with >= 10 objects and set them to green (value 2)
        blue_high_count_pixels = (vote_map == -1) & (actual_counts >= 10)
        vote_map_display[blue_high_count_pixels] = 2
    else:
        vote_map_display = vote_map
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create custom colormap based on show_mask
    if show_mask:
        # Extended colormap: blue (-1), white (0), red (1), green (2)
        colors_with_alpha = [
            to_rgba('blue', alpha=0.4),    # 30% opacity
            to_rgba('white', alpha=1.0),   # Fully opaque
            to_rgba('red', alpha=0.4),     # 30% opacity
            to_rgba('blue', alpha=1.0)    # Fully opaque
        ]
        cmap = ListedColormap(colors_with_alpha)
        vmin, vmax = -1, 2
    else:
        # Original colormap
        colors = ['blue', 'white', 'red']
        cmap = ListedColormap(colors)
        vmin, vmax = -1, 1
    
    # Plot voting map
    im = ax.imshow(vote_map_display.T, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], 
                   origin='lower', cmap=cmap, vmin=vmin, vmax=vmax, alpha=0.7, aspect='auto')
    
    # Add cluster objects if requested
    if show_cluster_objects:
        if full_catalog is None:
            print("Warning: show_cluster_objects=True but full_catalog not provided")
        else:
            # Filter for cluster objects
            cluster_mask = full_catalog['CLUSTER'] == cluster_name
            cluster_objects = full_catalog[cluster_mask]
            
            if len(cluster_objects) > 0:
                # Extract colors for cluster objects
                cluster_bg = cluster_objects['color_bg'].astype(float)
                cluster_ub = cluster_objects['color_ub'].astype(float)
                
                # Remove NaN values
                valid_cluster = ~(np.isnan(cluster_bg) | np.isnan(cluster_ub))
                cluster_bg = cluster_bg[valid_cluster]
                cluster_ub = cluster_ub[valid_cluster]
                
                # Plot cluster objects in grey
                ax.scatter(cluster_bg, cluster_ub, c='grey', alpha=0.5, s=10, 
                          label=f'{cluster_name} objects', edgecolors='none', zorder=1)
                
                print(f"Plotted {len(cluster_bg)} cluster objects from {cluster_name}")
            else:
                print(f"No objects found for cluster {cluster_name}")
    
    # Add redshift sequence members if requested
    if redshift_seq:
        if imposters_catalog is None or actual_catalog is None:
            print("Warning: redshift_seq=True but imposters_catalog or actual_catalog not provided")
        else:
            # Plot imposters
            if imposters_catalog is not None and len(imposters_catalog) > 0:
                imp_bg = imposters_catalog['color_bg'].astype(float)
                imp_ub = imposters_catalog['color_ub'].astype(float)
                imp_valid = ~(np.isnan(imp_bg) | np.isnan(imp_ub))
                imp_bg, imp_ub = imp_bg[imp_valid], imp_ub[imp_valid]
                
                ax.scatter(imp_bg, imp_ub, c='orange', alpha=1, s=15, 
                          label=f'Imposters ({len(imp_bg)})', edgecolors='black', zorder=3)
                print(f"Plotted {len(imp_bg)} imposter members")
            
            # Plot actual members
            if actual_catalog is not None and len(actual_catalog) > 0:
                act_bg = actual_catalog['color_bg'].astype(float)
                act_ub = actual_catalog['color_ub'].astype(float)
                act_valid = ~(np.isnan(act_bg) | np.isnan(act_ub))
                act_bg, act_ub = act_bg[act_valid], act_ub[act_valid]
                
                ax.scatter(act_bg, act_ub, c='magenta', alpha=1, s=15, 
                          label=f'Actual Members ({len(act_bg)})', edgecolors='black', zorder=3)
                print(f"Plotted {len(act_bg)} actual members")
    
    # Add object counts if requested
    if show_counts:
        # Get pixel centers
        x_centers = (x_edges[:-1] + x_edges[1:]) / 2
        y_centers = (y_edges[:-1] + y_edges[1:]) / 2
        count_sum = 0
        
        # Add text for pixels with enough objects
        for i, x in enumerate(x_centers):
            for j, y in enumerate(y_centers):
                # Note: when weighting=True, this shows weighted sum, not raw count
                count = int(total_objects[i, j]) if not weighting else f"{total_objects[i, j]:.1f}"
                
                # Decide whether to show count
                if blue_counts_only:
                    # Show count only if pixel is blue (vote_map == -1)
                    show_this_count = (vote_map[i, j] == -1) and (total_objects[i, j] > 0) and total_objects[i, j] >= 10
                    if show_this_count:
                        count_sum = count_sum + 1
                        
                else:
                    # Show count if above minimum threshold
                    show_this_count = (total_objects[i, j] >= min_count_display)
                    
                
                if show_this_count:
                    # Choose text color based on background
                    if show_mask and vote_map_display[i, j] == 2:  # Green
                        text_color = 'black'
                    else:
                        text_color = 'white' if vote_map[i, j] != 0 else 'black'
                    ax.text(x, y, str(count), ha='center', va='center', 
                           fontsize=8, color=text_color, weight='bold', zorder=4)
        print(count_sum)
    
    # Add scatter points if requested
    if show_scatter:
        train_bg = color_bg[training_mask] if training_mask is not None else color_bg
        train_ub = color_ub[training_mask] if training_mask is not None else color_ub
        train_z = redshift[training_mask] if training_mask is not None else redshift
        
        # Remove NaNs for plotting
        valid = ~(np.isnan(train_bg) | np.isnan(train_ub) | np.isnan(train_z))
        train_bg, train_ub, train_z = train_bg[valid], train_ub[valid], train_z[valid]
        
        background_mask = train_z > z_thresh
        foreground_mask = train_z <= z_thresh
        
        if np.any(background_mask):
            ax.scatter(train_bg[background_mask], train_ub[background_mask], 
                      c='red', alpha=0.3, s=3, label=f'Background (z > {z_thresh})', 
                      edgecolors='none', zorder=2)
        
        if np.any(foreground_mask):
            ax.scatter(train_bg[foreground_mask], train_ub[foreground_mask], 
                      c='blue', alpha=0.3, s=3, label=f'Foreground (z ≤ {z_thresh})', 
                      edgecolors='none', zorder=2)
    
    ax.set_xlabel('B - G Color')
    ax.set_ylabel('U - B Color')
    
    # Update title based on show_mask and weighting
    weight_text = " (weighted)" if weighting else ""
    if show_mask:
        ax.set_title('Unweighted Pixel Mask at Redshift 0.2') #f'{cluster_name}: Pixel Mask {weight_text}'
    else:
        ax.set_title(f'{cluster_name}: Pixel Voting Map{weight_text} (purity ≥ {purity_threshold:.0%})\n'
                     f'Blue = Foreground purity ≥ {purity_threshold:.0%}, Red = Otherwise')
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
        
    # Add statistics
    has_objects = total_objects > 0
    total_pixels = np.sum(has_objects)
    
    if show_mask:
        # Count mask pixels
        green_pixels = np.sum(vote_map_display == 2)
        blue_pixels = np.sum(vote_map == -1)  # Total blue pixels
        
        stats_text = f'Pixel Mask Statistics{weight_text} (size={pixel_size}):\n'
        stats_text += f'Total foreground pixels: {blue_pixels}\n'
        stats_text += f'Mask (blue with ≥10 objects): {green_pixels}'
    else:
        blue_pixels = np.sum(vote_map == -1)
        red_pixels = np.sum(vote_map == 1)
        
        # Calculate purity for statistics
        purity = np.zeros_like(fg_hist, dtype=float)
        purity[has_objects] = fg_hist[has_objects] / total_objects[has_objects]
        
        stats_text = f'Pixel Statistics{weight_text} (size={pixel_size}, purity threshold={purity_threshold:.0%}):\n'
        stats_text += f'Total pixels with objects: {total_pixels}\n'
        stats_text += f'Blue pixels (purity ≥ {purity_threshold:.0%}): {blue_pixels} ({blue_pixels/total_pixels*100:.1f}%)\n'
        stats_text += f'Red pixels (purity < {purity_threshold:.0%}): {red_pixels} ({red_pixels/total_pixels*100:.1f}%)'
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
            verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    if show_mask:
        print(f"\\nPixel Mask Analysis{weight_text}:")
        print(f"Pixel size: {pixel_size}")
        print(f"Total foreground pixels: {blue_pixels}")
        print(f" Mask (blue with ≥10 objects): {green_pixels}")
        if blue_pixels > 0:
            print(f"Percentage of blue pixels with ≥10 objects: {green_pixels/blue_pixels*100:.1f}%")
    else:
        print(f"\\nPixel Voting Analysis{weight_text} with Purity Threshold ({purity_threshold:.0%}):")
        print(f"Pixel size: {pixel_size}")
        print(f"Total pixels with objects: {total_pixels}")
        blue_pixels = np.sum(vote_map == -1)
        red_pixels = np.sum(vote_map == 1)
        print(f"Blue pixels (foreground purity ≥ {purity_threshold:.0%}): {blue_pixels} ({blue_pixels/total_pixels*100:.1f}%)")
        print(f"Red pixels (foreground purity < {purity_threshold:.0%}): {red_pixels} ({red_pixels/total_pixels*100:.1f}%)")
    
    if show_counts:
        count_type = "weighted sums" if weighting else "object counts"
        if blue_counts_only:
            print(f"\\nShowing {count_type} for blue pixels only")
        else:
            print(f"\\nShowing {count_type} for pixels with ≥ {min_count_display} objects")
    
    if show_mask:
        return vote_map_display, x_edges, y_edges, bg_hist, fg_hist, total_objects
    else:
        return vote_map, x_edges, y_edges, bg_hist, fg_hist, total_objects

In [ ]:
def apply_pixel_mask_to_catalog(vote_map_display, x_edges, y_edges, full_catalog, cluster_name, 
                               pixel_size, purity_threshold, show_mask=True, save_to="./", 
                               pixel_method_only=True, pixel_with_redseq=False, 
                               actual_catalog=None):
    """
    Apply the pixel mask to the full catalog and save foreground objects.
    
    Parameters
    ----------
    vote_map_display : 2D array
        The voting map with mask values (2=green, 3=magenta)
    x_edges, y_edges : array
        Bin edges for the pixel grid
    full_catalog : astropy.table.Table
        Full catalog containing all objects
    cluster_name : str
        Name of the cluster to filter for
    pixel_size : float
        Size of each pixel (for verification)
    show_mask : bool
        Whether the mask was applied (if False, use blue pixels instead)
    save_to : str
        Directory path where the FITS file will be saved (default: current directory)
    pixel_method_only : bool
        If True, save only pixel-selected objects (default: True)
    pixel_with_redseq : bool
        If True, include actual red sequence members in addition to pixel-selected
    actual_catalog : astropy.table.Table or None
        Catalog of actual red sequence members (required if pixel_with_redseq=True)
        
    Returns
    -------
    masked_catalog : astropy.table.Table
        Catalog containing selected foreground objects
    """
    
    # First, filter for the specific cluster
    cluster_mask = full_catalog['CLUSTER'] == cluster_name
    cluster_objects = full_catalog[cluster_mask]
    
    if len(cluster_objects) == 0:
        print(f"No objects found for cluster {cluster_name}")
        return Table()
    
    print(f"Found {len(cluster_objects)} objects for {cluster_name}")
    
    # Extract colors
    obj_bg = cluster_objects['color_bg'].astype(float)
    obj_ub = cluster_objects['color_ub'].astype(float)
    
    # Remove objects with NaN colors
    valid_colors = ~(np.isnan(obj_bg) | np.isnan(obj_ub))
    valid_indices = np.where(valid_colors)[0]
    
    obj_bg = obj_bg[valid_colors]
    obj_ub = obj_ub[valid_colors]
    
    print(f"Objects with valid colors: {len(obj_bg)}")
    
    # Initialize mask for objects to keep
    keep_objects = np.zeros(len(obj_bg), dtype=bool)
    
    # Check which objects fall into masked pixels
    for idx, (bg_val, ub_val) in enumerate(zip(obj_bg, obj_ub)):
        # Find pixel indices
        i = np.searchsorted(x_edges, bg_val) - 1
        j = np.searchsorted(y_edges, ub_val) - 1
        
        # Check bounds
        if 0 <= i < len(x_edges)-1 and 0 <= j < len(y_edges)-1:
            if show_mask:
                # Keep objects in green (2) or magenta (3) pixels
                if vote_map_display[i, j] in [2, 3]:
                    keep_objects[idx] = True
            else:
                # If no mask, keep objects in blue pixels (-1)
                if vote_map_display[i, j] == -1:
                    keep_objects[idx] = True
    
    # Apply the mask to get pixel-selected catalog
    final_indices = valid_indices[keep_objects]
    masked_catalog = cluster_objects[final_indices]
    
    # Add red sequence members if requested
    if pixel_with_redseq and actual_catalog is not None:
        # Filter actual catalog for this cluster if needed
        if 'CLUSTER' in actual_catalog.colnames:
            actual_cluster = actual_catalog[actual_catalog['CLUSTER'] == cluster_name]
        else:
            actual_cluster = actual_catalog
        
        # Combine catalogs, removing duplicates based on 'id' column
        if len(actual_cluster) > 0:
            combined_ids = set(masked_catalog['id'])
            new_members = actual_cluster[~np.isin(actual_cluster['id'], list(combined_ids))]
            masked_catalog = vstack([masked_catalog, new_members])
            print(f"Added {len(new_members)} unique red sequence members")
    elif pixel_with_redseq and actual_catalog is None:
        print("Warning: pixel_with_redseq=True but actual_catalog not provided")
    
    # Print summary statistics
    n_kept = len(masked_catalog)
    n_total = len(cluster_objects)
    
    if show_mask:
        # Count how many are in green vs magenta pixels (for pixel-only selection)
        n_green = 0
        n_magenta = 0
        
        for idx in np.where(keep_objects)[0]:
            bg_val = obj_bg[idx]
            ub_val = obj_ub[idx]
            i = np.searchsorted(x_edges, bg_val) - 1
            j = np.searchsorted(y_edges, ub_val) - 1
            
            if 0 <= i < len(x_edges)-1 and 0 <= j < len(y_edges)-1:
                if vote_map_display[i, j] == 2:
                    n_green += 1
                elif vote_map_display[i, j] == 3:
                    n_magenta += 1
        
        print(f"\nMask Application Results:")
        print(f"Total {cluster_name} objects: {n_total}")
        print(f"Objects in masked pixels: {np.sum(keep_objects)} ({np.sum(keep_objects)/n_total*100:.1f}%)")
        print(f"  - In green pixels: {n_green}")
        print(f"  - In magenta pixels: {n_magenta}")
        if pixel_with_redseq:
            print(f"Total foreground objects (including red sequence): {n_kept}")
    else:
        print(f"\nBlue Pixel Selection Results:")
        print(f"Total {cluster_name} objects: {n_total}")
        print(f"Objects in blue pixels: {n_kept} ({n_kept/n_total*100:.1f}%)")
    
    # Save to file
    import os
    if pixel_with_redseq:
        filename = f'{cluster_name}_{PURITY_THRESHOLD}_foreground_pixel_{PIXEL_SIZE}_and_redseq.fits'
    else:
        filename = f'{cluster_name}_{PURITY_THRESHOLD}_foreground_pixel_{PIXEL_SIZE}.fits'
    filepath = os.path.join(save_to, filename)
    masked_catalog.write(filepath, overwrite=True)
    print(f"\nSaved {len(masked_catalog)} foreground galaxies to {filepath}")
    
    return masked_catalog

In [ ]:
def spatial_plot(fits_file, vote_map_display, x_edges, y_edges, cluster_name, 
                 include_members=False, actual_catalog=None, figsize=(16, 6)):
    """
    Plot spatial distribution with scatter and density views.
    
    Parameters
    ----------
    fits_file : str
        Path to foreground catalog FITS file
    vote_map_display : 2D array
        Voting map for pixel classification
    x_edges, y_edges : array
        Bin edges for pixel classification
    cluster_name : str
        Cluster name for title
    include_members : bool
        If True, overlay actual cluster members
    actual_catalog : Table or str
        Actual members catalog (required if include_members=True)
    """
    
    # Load foreground catalog
    foreground = Table.read(fits_file)
    fg_ra = foreground['ra'].data.astype(float)
    fg_dec = foreground['dec'].data.astype(float)
    fg_bg = foreground['color_bg'].data.astype(float)
    fg_ub = foreground['color_ub'].data.astype(float)
    
    # Classify each object as green or magenta
    green_mask = np.zeros(len(fg_ra), dtype=bool)
    magenta_mask = np.zeros(len(fg_ra), dtype=bool)
    
    for idx in range(len(fg_ra)):
        i = np.searchsorted(x_edges, fg_bg[idx]) - 1
        j = np.searchsorted(y_edges, fg_ub[idx]) - 1
        
        if 0 <= i < len(x_edges)-1 and 0 <= j < len(y_edges)-1:
            if vote_map_display[i, j] == 2:
                green_mask[idx] = True
            elif vote_map_display[i, j] == 3:
                magenta_mask[idx] = True
    
    # Load actual catalog if needed
    if include_members:
        if isinstance(actual_catalog, str):
            actual = Table.read(actual_catalog)
        else:
            actual = actual_catalog
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Left panel: Scatter plot
    ax1.scatter(fg_ra[green_mask], fg_dec[green_mask], c='green', alpha=0.6, s=20,
               label=f'Green pixels ({np.sum(green_mask)})', edgecolors='darkgreen', linewidth=0.5)
    ax1.scatter(fg_ra[magenta_mask], fg_dec[magenta_mask], c='magenta', alpha=0.8, s=30,
               label=f'Magenta pixels ({np.sum(magenta_mask)})', edgecolors='purple', linewidth=0.5)
    
    if include_members and actual is not None:
        ax1.scatter(actual['ra'].data, actual['dec'].data, c='purple', s=100, marker='*',
                   label=f'Actual Members ({len(actual)})', 
                   edgecolors='black', linewidth=1.5)
    
    ax1.set_xlabel('RA (degrees)')
    ax1.set_ylabel('Dec (degrees)')
    title = f'{cluster_name}: Foreground vs Actual Members' if include_members else f'{cluster_name}: Spatial Distribution of Foreground Galaxies'
    ax1.set_title(title)
    ax1.invert_xaxis()
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Right panel: Density map
    if include_members and actual is not None:
        # Combine foreground (green+magenta) with actual members
        mask_combined = green_mask | magenta_mask
        all_ra = np.concatenate([fg_ra[mask_combined], actual['ra'].data])
        all_dec = np.concatenate([fg_dec[mask_combined], actual['dec'].data])
    else:
        # Only green and magenta pixels
        mask_combined = green_mask | magenta_mask
        all_ra = fg_ra[mask_combined]
        all_dec = fg_dec[mask_combined]
    
    ra_range = [all_ra.min(), all_ra.max()]
    dec_range = [all_dec.min(), all_dec.max()]
    
    bins_ra = int((ra_range[1] - ra_range[0]) / 0.005)  
    bins_dec = int((dec_range[1] - dec_range[0]) / 0.005)
    
    H, ra_edges_hist, dec_edges_hist = np.histogram2d(all_ra, all_dec, 
                                                      bins=[bins_ra, bins_dec],
                                                      range=[ra_range, dec_range])
    
    from scipy.ndimage import gaussian_filter
    H_smooth = gaussian_filter(H, sigma=1.5)
    
    extent = [ra_edges_hist[0], ra_edges_hist[-1], dec_edges_hist[0], dec_edges_hist[-1]]
    im = ax2.imshow(H_smooth.T, extent=extent, origin='lower', 
                    cmap='plasma', aspect='equal', interpolation='bilinear')
    
    cbar = plt.colorbar(im, ax=ax2, label='Count')
    
    ax2.set_xlabel('RA (degrees)')
    ax2.set_ylabel('Dec (degrees)')
    ax2.set_title(f'Resolution: 0.5 arcmin, kernel: 1.5')
    ax2.invert_xaxis()
    ax2.set_xlim(ra_range[::-1])  # reverse ra order
    ax2.set_ylim(dec_range)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Loaded {len(foreground)} foreground galaxies")
    print(f"Green pixels: {np.sum(green_mask)}, Magenta pixels: {np.sum(magenta_mask)}")
    
    if include_members:
        return fg_ra[green_mask], fg_dec[green_mask], fg_ra[magenta_mask], fg_dec[magenta_mask], actual
    else:
        return fg_ra[green_mask], fg_dec[green_mask], fg_ra[magenta_mask], fg_dec[magenta_mask]

In [ ]:
def plot_radial_distribution_rb(fits_file, cluster_name, actual_catalog=None, figsize=(10, 6)):
    """Plot radial distribution of foreground galaxies and actual members overlaid."""
    
    # Read foreground catalog
    cat = Table.read(fits_file)
    r_b_pixel = cat['R_b'].astype(float)
    r_b = r_b_pixel * 0.141
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot foreground galaxies first (behind)
    n1, bins, patches1 = ax.hist(r_b, bins=30, alpha=0.6, color='blue', 
                                 edgecolor='black', label=f'Foreground ({len(cat)})')
    
    # Plot actual members if provided (in front)
    if actual_catalog is not None:
        actual_r_b_pixel = actual_catalog['R_b'].astype(float)
        actual_r_b = actual_r_b_pixel * 0.141
        
        n2, bins, patches2 = ax.hist(actual_r_b, bins=bins, alpha=0.7, color='magenta', 
                                     edgecolor='black', label=f'Actual Members ({len(actual_catalog)})')
        
        # Add median lines
        ax.axvline(np.median(r_b), color='darkblue', linestyle='--', linewidth=2,
                  label=f'Foreground Median: {np.median(r_b):.2f}')
        ax.axvline(np.median(actual_r_b), color='darkmagenta', linestyle='--', linewidth=2,
                  label=f'Actual Median: {np.median(actual_r_b):.2f}')
    else:
        ax.axvline(np.median(r_b), color='red', linestyle='--', linewidth=2,
                  label=f'Median: {np.median(r_b):.2f}')
    
    ax.set_xlabel('R_b Radius (arcsec)')
    ax.set_ylabel('Number of Objects')
    ax.set_title(f'{cluster_name}: Radial Distribution Comparison')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    if actual_catalog is not None:
        print(f"Foreground median radius: {np.median(r_b):.2f} arcsec")
        print(f"Actual members median radius: {np.median(actual_r_b):.2f} arcsec")
        return r_b, actual_r_b
    else:
        return r_b

In [ ]:
def create_pixel_error_maps(color_bg, color_ub, color_bg_err, color_ub_err, 
                           xlim, ylim, pixel_size=0.1, training_mask=None, 
                           statistic='median'):
    """
    Create pixel-based maps showing statistical summary of color errors in each pixel.
    
    Parameters
    ----------
    color_bg, color_ub : array-like
        B-G and U-B colors
    color_bg_err, color_ub_err : array-like
        Color uncertainties
    xlim, ylim : tuple
        Plot limits for color axes
    pixel_size : float
        Size of each pixel in color space
    training_mask : boolean array, optional
        Mask for high-quality training data
    statistic : str
        'median', 'max', or 'weighted' - how to summarize errors in each pixel
        
    Returns
    -------
    bg_error_map : 2D array
        Statistical summary of B-G error per pixel
    ub_error_map : 2D array
        Statistical summary of U-B error per pixel
    counts_map : 2D array
        Number of objects per pixel
    x_edges, y_edges : array
        Bin edges for the pixel grid
    """
    
    if training_mask is not None:
        color_bg = color_bg[training_mask]
        color_ub = color_ub[training_mask]
        color_bg_err = color_bg_err[training_mask]
        color_ub_err = color_ub_err[training_mask]
    
    # Remove NaN values
    valid_mask = ~(np.isnan(color_bg) | np.isnan(color_ub) | 
                   np.isnan(color_bg_err) | np.isnan(color_ub_err))
    color_bg = color_bg[valid_mask]
    color_ub = color_ub[valid_mask]
    color_bg_err = color_bg_err[valid_mask]
    color_ub_err = color_ub_err[valid_mask]
    
    # Create pixel grid
    x_edges = np.arange(xlim[0], xlim[1] + pixel_size, pixel_size)
    y_edges = np.arange(ylim[0], ylim[1] + pixel_size, pixel_size)
    
    # Initialize lists to store error values for each pixel
    bg_error_lists = [[[] for _ in range(len(y_edges)-1)] for _ in range(len(x_edges)-1)]
    ub_error_lists = [[[] for _ in range(len(y_edges)-1)] for _ in range(len(x_edges)-1)]
    
    # Bin the data
    for i in range(len(color_bg)):
        # Find which pixel this object belongs to
        x_idx = np.searchsorted(x_edges, color_bg[i]) - 1
        y_idx = np.searchsorted(y_edges, color_ub[i]) - 1
        
        # Check bounds
        if (0 <= x_idx < len(x_edges)-1) and (0 <= y_idx < len(y_edges)-1):
            bg_error_lists[x_idx][y_idx].append(color_bg_err[i])
            ub_error_lists[x_idx][y_idx].append(color_ub_err[i])
    
    # Compute statistics
    bg_error_map = np.full((len(x_edges)-1, len(y_edges)-1), np.nan)
    ub_error_map = np.full((len(x_edges)-1, len(y_edges)-1), np.nan)
    counts_map = np.zeros((len(x_edges)-1, len(y_edges)-1))
    
    epsilon = 1e-10  # Small value to avoid division by zero
    
    for i in range(len(x_edges)-1):
        for j in range(len(y_edges)-1):
            if bg_error_lists[i][j]:  # If there are any values
                bg_errors = np.array(bg_error_lists[i][j])
                ub_errors = np.array(ub_error_lists[i][j])
                
                if statistic == 'max':
                    bg_error_map[i, j] = np.max(bg_errors)
                    ub_error_map[i, j] = np.max(ub_errors)
                elif statistic == 'median':
                    bg_error_map[i, j] = np.median(bg_errors)
                    ub_error_map[i, j] = np.median(ub_errors)
                elif statistic == 'weighted':
                    # Calculate weighted average where weight = 1/error²
                    # For BG errors
                    bg_weights = 1.0 / (bg_errors**2 + epsilon)
                    bg_error_map[i, j] = np.sum(bg_weights * bg_errors) / np.sum(bg_weights)
                    
                    # For UB errors
                    ub_weights = 1.0 / (ub_errors**2 + epsilon)
                    ub_error_map[i, j] = np.sum(ub_weights * ub_errors) / np.sum(ub_weights)
                else:
                    raise ValueError(f"Invalid statistic: {statistic}. Use 'median', 'max', or 'weighted'")
                
                counts_map[i, j] = len(bg_error_lists[i][j])
    
    return bg_error_map, ub_error_map, counts_map, x_edges, y_edges

In [ ]:
def plot_pixel_error_maps(color_bg, color_ub, color_bg_err, color_ub_err,
                         xlim, ylim, training_mask, cluster_name, 
                         pixel_size=0.1, vmax_percentile=95, statistic='median', 
                         figsize=(14, 6)):
    """
    Plot pixel-based error maps for both B-G and U-B colors using specified statistic.
    
    Parameters
    ----------
    color_bg, color_ub : array-like
        B-G and U-B colors
    color_bg_err, color_ub_err : array-like
        Color uncertainties
    xlim, ylim : tuple
        Plot limits for color axes
    training_mask : boolean array
        Mask for high-quality training data
    cluster_name : str
        Name for plot title
    pixel_size : float
        Size of each pixel in color space
    vmax_percentile : float
        Percentile for colorbar maximum (to handle outliers)
    statistic : str
        'median', 'max', or 'weighted' - how to summarize errors
    figsize : tuple
        Figure size
    """
    
    # Create error maps
    bg_error_map, ub_error_map, counts_map, x_edges, y_edges = create_pixel_error_maps(
        color_bg, color_ub, color_bg_err, color_ub_err, 
        xlim, ylim, pixel_size, training_mask, statistic
    )
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # Define color limits based on percentiles (to handle outliers)
    bg_vmax = np.nanpercentile(bg_error_map, vmax_percentile)
    ub_vmax = np.nanpercentile(ub_error_map, vmax_percentile)
    
    # Determine metric label
    if statistic == 'max':
        metric = "Maximum"
    elif statistic == 'median':
        metric = "Median"
    elif statistic == 'weighted':
        metric = "Weighted Average"
    
    # Plot B-G error map
    im1 = ax1.imshow(bg_error_map.T, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], 
                     origin='lower', cmap='viridis', vmin=0, vmax=bg_vmax, 
                     aspect='equal', interpolation='nearest')
    ax1.set_xlabel('B - G Color')
    ax1.set_ylabel('U - B Color')
    ax1.set_title(f'All Clusters {metric} B-G Color Error per Pixel')
    ax1.grid(True, alpha=0.3)
    cbar1 = plt.colorbar(im1, ax=ax1, label=f'{metric} B-G Error')
    
    # Plot U-B error map
    im2 = ax2.imshow(ub_error_map.T, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], 
                     origin='lower', cmap='viridis', vmin=0, vmax=ub_vmax, 
                     aspect='equal', interpolation='nearest')
    ax2.set_xlabel('B - G Color')
    ax2.set_ylabel('U - B Color')
    ax2.set_title(f'All Clusters {metric} U-B Color Error per Pixel')
    ax2.grid(True, alpha=0.3)
    cbar2 = plt.colorbar(im2, ax=ax2, label=f'{metric} U-B Error')
    
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nPixel Error Map Analysis ({metric} mode):")
    print(f"Pixel size: {pixel_size}")
    print(f"Total pixels with objects: {np.sum(counts_map > 0)}")
    print(f"\nB-G Error Statistics ({metric}):")
    
    
    if statistic == 'weighted':
        print("\nNote: Weighted average gives more weight to precise measurements (1/error²)")
    
    return bg_error_map, ub_error_map, counts_map

In [ ]:
def plot_combined_error_map(color_bg, color_ub, color_bg_err, color_ub_err,
                           xlim, ylim, training_mask, cluster_name, 
                           pixel_size=0.1, vmax_percentile=95, statistic='median', 
                           figsize=(10, 8)):
    """
    Plot a single map showing combined color error.
    
    Parameters
    ----------
    Same as plot_pixel_error_maps
    """
    
    # Create error maps
    bg_error_map, ub_error_map, counts_map, x_edges, y_edges = create_pixel_error_maps(
        color_bg, color_ub, color_bg_err, color_ub_err, 
        xlim, ylim, pixel_size, training_mask, statistic
    )
    
    # Compute combined error (RMS of errors)
    combined_error = np.sqrt(bg_error_map**2 + ub_error_map**2)
    
    # Determine metric label
    if statistic == 'max':
        metric = "Maximum"
    elif statistic == 'median':
        metric = "Median"
    elif statistic == 'weighted':
        metric = "Weighted Average"
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize)
    
    # Define color limit
    vmax = np.nanpercentile(combined_error, vmax_percentile)
    
    # Plot combined error map
    im = ax.imshow(combined_error.T, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], 
                   origin='lower', cmap='viridis', vmin=0, vmax=vmax, 
                   aspect='equal', interpolation='nearest')
    ax.set_xlabel('B - G Color')
    ax.set_ylabel('U - B Color')
    ax.set_title(f'{cluster_name}: Combined Color Error per Pixel (RMS of {metric}s)')
    ax.grid(True, alpha=0.3)
    cbar = plt.colorbar(im, ax=ax, label='Combined Error (RMS)')
    
    # Add statistics
    valid_combined = combined_error[~np.isnan(combined_error)]
    
    stats_text = f'Combined Error Statistics:\n'
    stats_text += f'Min: {np.min(valid_combined):.3f}\n'
    stats_text += f'Max: {np.max(valid_combined):.3f}\n'
    stats_text += f'Mean: {np.mean(valid_combined):.3f}\n'
    stats_text += f'Median: {np.median(valid_combined):.3f}'
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    return combined_error

## Plots

### Make your Map

In [ ]:
vote_map, x_edges, y_edges, bg_hist, fg_hist, total_objects = plot_pixel_voting_map(
    color_bg, color_ub, redshift, Z_THRESH, XLIM, YLIM,
    training_mask, CLUSTER_NAME,
    PIXEL_SIZE,
    PURITY_THRESHOLD,
    show_scatter=False,
    show_counts=False,
    show_mask=True,
    actual_catalog=Abell3411_actual,
    redshift_seq=False,
    imposters_catalog=Abell3411_imposters,
    figsize=(10, 8),
    weighting=False,             # Enable weighted purity
    color_bg_err=color_bg_err,   # B-G color errors
    color_ub_err=color_ub_err    # U-B color errors
)

In [ ]:
plot_pixel_error_maps(
    color_bg_full, color_ub_full, color_bg_err_full, color_ub_err_full,
    XLIM, YLIM, training_mask=None, cluster_name = CLUSTER_NAME,
    pixel_size=0.1,
    vmax_percentile=85,  
    statistic='weighted',  
    figsize=(14, 6)
)

## Write to File 

In [ ]:

foreground_galaxies = apply_pixel_mask_to_catalog(
    vote_map_display=vote_map,
    x_edges=x_edges,
    y_edges=y_edges,
    full_catalog=full_cat,
    cluster_name=CLUSTER_NAME,
    pixel_size=PIXEL_SIZE,
    purity_threshold=PURITY_THRESHOLD,
    show_mask=True,
    save_to=f"/projects/mccleary_group/amit.m/color_fg_data/{CLUSTER_NAME}/foreground/test",
    pixel_with_redseq=True,
    actual_catalog=Abell3411_actual
)

## Extra Stuff with the Fits File

### Plot the foreground objects in RADEC space

In [ ]:
result = spatial_plot('/projects/mccleary_group/amit.m/color_fg_data/1E0657_Bullet/foreground/1E0657_Bullet_0.75_foreground_pixel_0.05_and_redseq.fits', 
             vote_map, x_edges, y_edges, CLUSTER_NAME, include_members=False, actual_catalog = Abell3411_actual)

## Plot the Radius Distribution of the Foreground Objects 

In [ ]:

r_b_values = plot_radial_distribution_rb(
    '/projects/mccleary_group/amit.m/color_fg_data/1E0657_Bullet/foreground/1E0657_Bullet_0.75_foreground_pixel_0.05_and_redseq.fits',
    '1E0657_Bullet',
    actual_catalog=Bullet_actual
)